# A Recommender System Using >>Apache Spark 2.4
### Predictive Analytics
#### Licence:
You can use this code for anything you may wish only leave this page:
#### AS IS; HOW IS, WHERE IS

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://192.168.8.130:7077').appName('Recommender System').getOrCreate()

### Reading the Dataset

In [25]:
df = spark.read.csv('./static/movie_ratings_df.csv', inferSchema=True, header=True)
df.show(10, True) # Pulls the fist ten rows of the dataset

+------+------------+------+
|userId|       title|rating|
+------+------------+------+
|   196|Kolya (1996)|     3|
|    63|Kolya (1996)|     3|
|   226|Kolya (1996)|     5|
|   154|Kolya (1996)|     3|
|   306|Kolya (1996)|     5|
|   296|Kolya (1996)|     4|
|    34|Kolya (1996)|     5|
|   271|Kolya (1996)|     4|
|   201|Kolya (1996)|     4|
|   209|Kolya (1996)|     4|
+------+------------+------+
only showing top 10 rows



### Data size in terms of rows and records

In [26]:
print((df.count(), len(df.columns)))

(100000, 3)


Our Dataset contains 100,000 records with only three columns

Data Schema

In [27]:
df.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- rating: integer (nullable = true)



#### Two of the columns are Numerical while title column is categorical

#### Top users by number of moviers rated

In [28]:
df.groupBy('userId').count().orderBy('count', ascending=False).show(10, False)

+------+-----+
|userId|count|
+------+-----+
|405   |737  |
|655   |685  |
|13    |636  |
|450   |540  |
|276   |518  |
|416   |493  |
|537   |490  |
|303   |484  |
|234   |480  |
|393   |448  |
+------+-----+
only showing top 10 rows



#### Bottom users by number of moviers rated

In [29]:
df.groupBy('userId').count().orderBy('count', ascending=True).show(10, False)

+------+-----+
|userId|count|
+------+-----+
|732   |20   |
|636   |20   |
|572   |20   |
|685   |20   |
|631   |20   |
|34    |20   |
|596   |20   |
|926   |20   |
|93    |20   |
|300   |20   |
+------+-----+
only showing top 10 rows



## Feature Engineering

In [30]:
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer, IndexToString

#### So now we create the stringindexer object by mentioning the input column and output column.

In [31]:
stringIndexer = StringIndexer(inputCol='title',outputCol='New_Title')
model = stringIndexer.fit(df)
indexed = model.transform(df)
indexed.show(10, False)

+------+------------+------+---------+
|userId|title       |rating|New_Title|
+------+------------+------+---------+
|196   |Kolya (1996)|3     |287.0    |
|63    |Kolya (1996)|3     |287.0    |
|226   |Kolya (1996)|5     |287.0    |
|154   |Kolya (1996)|3     |287.0    |
|306   |Kolya (1996)|5     |287.0    |
|296   |Kolya (1996)|4     |287.0    |
|34    |Kolya (1996)|5     |287.0    |
|271   |Kolya (1996)|4     |287.0    |
|201   |Kolya (1996)|4     |287.0    |
|209   |Kolya (1996)|4     |287.0    |
+------+------------+------+---------+
only showing top 10 rows



In [32]:
print (df.groupBy('title').count().orderBy('count',ascending=False).show(10,False),
       indexed.groupBy('New_Title').count().orderBy('count',ascending=False).show(10,False))

+-----------------------------+-----+
|title                        |count|
+-----------------------------+-----+
|Star Wars (1977)             |583  |
|Contact (1997)               |509  |
|Fargo (1996)                 |508  |
|Return of the Jedi (1983)    |507  |
|Liar Liar (1997)             |485  |
|English Patient, The (1996)  |481  |
|Scream (1996)                |478  |
|Toy Story (1995)             |452  |
|Air Force One (1997)         |431  |
|Independence Day (ID4) (1996)|429  |
+-----------------------------+-----+
only showing top 10 rows

+---------+-----+
|New_Title|count|
+---------+-----+
|0.0      |583  |
|1.0      |509  |
|2.0      |508  |
|3.0      |507  |
|4.0      |485  |
|5.0      |481  |
|6.0      |478  |
|7.0      |452  |
|8.0      |431  |
|9.0      |429  |
+---------+-----+
only showing top 10 rows

None None


## Pliting The Data Set
We split it into a 80 to 20 ratio to train the model and test its accuracy

In [33]:
train,test=indexed.randomSplit([0.8,0.2])
print("Training set", train.count())
print("Testing set", test.count())

Training set 80075
Testing set 19925


##  Building and Training the Model

In [34]:
from pyspark.ml.recommendation import ALS
rec=ALS(maxIter=10,regParam=0.01,userCol='userId',itemCol='New_Title',ratingCol='rating',nonnegative=True)
rec_model = rec.fit(train)

### Performance evaluation on our test data
Here we will chack the performance of our model on unseen data.

In [35]:
predict_ratings = rec_model.transform(test)
predict_ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- New_Title: double (nullable = false)
 |-- prediction: float (nullable = false)



In [36]:
predict_ratings.show(10)

+------+--------------------+------+---------+----------+
|userId|               title|rating|New_Title|prediction|
+------+--------------------+------+---------+----------+
|   593|Much Ado About No...|     5|    148.0|  4.461378|
|   497|Much Ado About No...|     2|    148.0|  4.042518|
|   844|Much Ado About No...|     5|    148.0|  4.516927|
|   707|Much Ado About No...|     3|    148.0| 4.3230314|
|   660|Much Ado About No...|     3|    148.0|   3.16418|
|     1|Much Ado About No...|     3|    148.0| 4.4556293|
|   429|Much Ado About No...|     4|    148.0|  4.284988|
|   292|Much Ado About No...|     5|    148.0| 4.4326243|
|    96|Much Ado About No...|     3|    148.0|  4.192071|
|   452|Much Ado About No...|     3|    148.0|  4.091294|
+------+--------------------+------+---------+----------+
only showing top 10 rows



In [37]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(metricName = 'rmse', predictionCol = 'prediction', labelCol = 'rating')
rmse = evaluator.evaluate(predict_ratings)
print(rmse)

nan


##### The rmse is non we can equate this to zero meaning there is no error.

### Recommend Top Movies That Active User Might Like
The first step is to create a list of unique movies in the dataframe.

In [38]:
unique_movies=indexed.select('New_Title').distinct()
unique_movies.count() ## Total numbe rof individual movies in our new dataframe

1664

In [39]:
a = unique_movies.alias('a')

#### We can select any user within the dataset for which we need to recommend other movies. In our case, we go ahead with userId = 96.

#### We will filter the movies that this active user has already rated or seen.

In [40]:
user_id=96
watched_movies=indexed.filter(indexed['userId'] == user_id).select('New_Title').distinct()
watched_movies.count() # Number of movies the user has watched

56

In [41]:
b=watched_movies.alias('b')

#### So, there are total of 56 unique movies out of 1,664 movies that this active user has already rated. So, we would want to recommend movies from the remaining 1608 movies.

In [42]:
total_movies = a.join(b, a.New_Title == b.New_Title,how='left')
total_movies.show()

+---------+---------+
|New_Title|New_Title|
+---------+---------+
|    305.0|     null|
|    596.0|     null|
|    299.0|     null|
|    769.0|     null|
|    692.0|     null|
|    934.0|     null|
|   1051.0|     null|
|    496.0|     null|
|    558.0|     null|
|    170.0|     null|
|    184.0|     null|
|    576.0|     null|
|    147.0|     null|
|    810.0|     null|
|    720.0|     null|
|    782.0|     null|
|   1369.0|     null|
|   1587.0|     null|
|    160.0|     null|
|    608.0|     null|
+---------+---------+
only showing top 20 rows



In [43]:
remaining_movies=total_movies.where(col("b.New_Title").isNull()).select(a.New_Title).distinct()
remaining_movies.count()

1608

In [44]:
remaining_movies=remaining_movies.withColumn("userId",lit(int(user_id)))
remaining_movies.show(10, False)

+---------+------+
|New_Title|userId|
+---------+------+
|305.0    |96    |
|596.0    |96    |
|299.0    |96    |
|769.0    |96    |
|692.0    |96    |
|934.0    |96    |
|1051.0   |96    |
|496.0    |96    |
|558.0    |96    |
|170.0    |96    |
+---------+------+
only showing top 10 rows



### Finally, we can now make the predictions on this remaining movie’s dataset for the active user using the recommender model that we built earlier. We filter only a few top recommendations that have the highest predicted ratings.

In [45]:
recommendations = rec_model.transform(remaining_movies).orderBy('prediction', ascending = False)
recommendations.show(100, False)

+---------+------+----------+
|New_Title|userId|prediction|
+---------+------+----------+
|1572.0   |96    |NaN       |
|1483.0   |96    |NaN       |
|1577.0   |96    |NaN       |
|1631.0   |96    |NaN       |
|1554.0   |96    |NaN       |
|1613.0   |96    |NaN       |
|1567.0   |96    |NaN       |
|1574.0   |96    |NaN       |
|1585.0   |96    |NaN       |
|1575.0   |96    |NaN       |
|1589.0   |96    |NaN       |
|1544.0   |96    |NaN       |
|1591.0   |96    |NaN       |
|1400.0   |96    |NaN       |
|1602.0   |96    |NaN       |
|1660.0   |96    |NaN       |
|1461.0   |96    |NaN       |
|1601.0   |96    |NaN       |
|1568.0   |96    |NaN       |
|1549.0   |96    |NaN       |
|1624.0   |96    |NaN       |
|1536.0   |96    |NaN       |
|1644.0   |96    |NaN       |
|1600.0   |96    |NaN       |
|1557.0   |96    |NaN       |
|1560.0   |96    |NaN       |
|1540.0   |96    |NaN       |
|1558.0   |96    |NaN       |
|1466.0   |96    |NaN       |
|1347.0   |96    |9.452741  |
|1370.0   

### Let us add the movie title to the recommendations

In [46]:
movie_title = IndexToString(inputCol="New_Title",outputCol="title",labels=model.labels)
final_recommendations=movie_title.transform(recommendations)
final_recommendations.show(100,False)

+---------+------+----------+-----------------------------------------------------------+
|New_Title|userId|prediction|title                                                      |
+---------+------+----------+-----------------------------------------------------------+
|1549.0   |96    |NaN       |Dadetown (1995)                                            |
|1577.0   |96    |NaN       |Hungarian Fairy Tale, A (1987)                             |
|1568.0   |96    |NaN       |Girls Town (1996)                                          |
|1544.0   |96    |NaN       |Careful (1992)                                             |
|1540.0   |96    |NaN       |Bloody Child, The (1996)                                   |
|1536.0   |96    |NaN       |Baton Rouge (1988)                                         |
|1466.0   |96    |NaN       |Babyfever (1994)                                           |
|1461.0   |96    |NaN       |Wedding Gift, The (1994)                                   |
|1560.0   

### Thats it guys
### A simple collaborative filtering based recommender system in PySpark using the ALS method to recommend movies to the users

### Bye

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("My First Spark App")
    .getOrCreate()